In [2]:
# Chargement du jeu de données
import pandas as pd
df = pd.read_csv('data_projet.csv', sep=';', encoding='utf8')

FileNotFoundError: [Errno 2] No such file or directory: 'data_projet.csv'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# affichage des principales caractéristiques du jeu de données
display(df.shape)
display(df.head(5))
display(df.info())
display(df.describe())
display(df.describe(include='object'))

In [ ]:
# affichage des valeurs uniques pour les colonnes 'company', 'source' et 'maj'
print(df['company'].unique())
print(df['source'].unique())
print(df['maj'].unique())

In [ ]:
# 2 company étudiées ShowRoom et VeePee
# 2 sites stockent des commentaires : TrustPilot et TrustedShop

In [ ]:
print(df['source'][df['company']=='ShowRoom'].unique())
print(df['source'][df['company']=='VeePee'].unique())
# quand company = ShowRoom les avis peuvent être fait sur TrustPilot et TrustedShop
# quand company = VeePee les avis peuvent être fait sur TrustPilot

In [ ]:
# calcul des % de NaN de chaque colonne 
colonne = df.columns
NaN = [round((df[i].isna().sum()/len(df))*100, 2) for i in colonne]
NaN

In [ ]:
# association des % aux noms de variable dans un dictionnaire
result_NaN = {}
for i, j in zip(colonne, NaN):
    result_NaN[i] = j

In [ ]:
result_NaN

In [ ]:
# affichage des % de NaN dans un DataFrame
df_NaN = pd.DataFrame.from_dict(result_NaN, orient='index')
df_NaN

In [ ]:
# intuitivement il semble que les colonnes date commande et écart soient renseignés ensemble
# et qu'ils soient renseignés pour une seule company. On peut le vérifier
print(df[['source', 'company']][df['date_commande'].isna()==False].value_counts())
print(df[['source', 'company']][df['ecart'].isna()==False].value_counts())

In [ ]:
# on peut aussi procéder par un groupby qui permet de voir un peu mieux la répartition des dates de commande et des écarts
df.groupby(by=['source', 'company']).agg({'date_commande' : 'count', 'star' : 'count'})

In [ ]:
# les colonnes date_commande et écart ne semble pas apporter grand chose. on va toute de même les garder car il est possible
# qu'ils y ait une corrélation entre le commentaire, la star et l'écart

In [ ]:
# les colonnes maj et date_commande n'ayant que peut d'intérêt (car trop de valeur NaN pour maj et ecart suffit pour date_commande)
# on peut donc supprimer ces 2 colonnes
# les lignes sans commentaires peuvent être supprimées car ne représentent que 0.15% du DataSet
# les lignes sans date peuvent être supprimées car elles ne représentent que 1.89% du data set

In [ ]:
df_clean1 = df.drop(columns=['maj', 'date_commande'], axis=1)
df_clean1 = df_clean1.dropna(subset=['Commentaire', 'date'])

In [ ]:
df_clean1['date'][df_clean1['date'].str.contains('/2019')]

In [ ]:
# dans la colonne date une partie des données est au format dd/mm/yyyy et l'autre partie est au format yyyy-mm-dd
# nous allons donc convertir les données qui sont au format dd/mm/yyyy au format yyyy-mm-dd

df_clean1['date'][df_clean1['date'].str.contains('-')==False] = df_clean1['date'][df_clean1['date'].str.contains('-')==False].apply(lambda x: x[6:10]+'-'+x[3:5]+'-'+x[0:2])

In [ ]:
# on vérifie que toutes les données ont bien été transformées
df_clean1['date'][df_clean1['date'].str.contains('/')==True]

In [ ]:
# on les mets sur 10 caractères pour que l'on puisse les trasnformer à un format datetime
df_clean1['date'] = df_clean1['date'].apply(lambda x : x[0:11])

# puis on les convertit à un format datetime
df_clean1['date'] = pd.to_datetime(df_clean1['date'])

In [ ]:
#on vérifie le nb de données et leur type
df_clean1.info()

In [ ]:
# on calcule le min est le max pour avoir la plage de date couverte
print("la date min des commentaires est : ", df_clean1['date'].min(), "\nla date max des commentaires est : ",df_clean1['date'].max())

In [ ]:
# on rajoute une colonne avec seulement l'année du commentaire ce qui permettra d'étudier plus facilement les commentaires 
df_clean1['year'] = df_clean1['date'].dt.year

In [ ]:
# distribution des données en fonction des sources, company et années 
display(df_clean1.groupby(by=['source', 'company', 'year']).agg({'star' : 'count'}))
print("\nOn constate que TrustedShop n'a fourni des commentaires qu'à partir de 2020 alors que TrustPilot a commencé en 2015 pour le site VeePee et en 2019 pour le site ShowRoom")

In [ ]:
# on regarde la répartition des commentaires sur les différentes années
print("la distribution des commentaires par année es la suivante :")
print(df_clean1['year'].value_counts())

# on peut afficher les résultats de la distribution des commentaires par année dans un graphique
sns.countplot(x='year', data=df_clean1, palette='deep', hue='company')
plt.legend((), loc='upper left');

print("\nOnconstate que la majorité des commentaires on été réalisés à partir de 2020 grace à l'utilisation de TrustedShop\n")

In [ ]:
# on peut aussi étudier la répartition de l'attribution des étoiles selon les company, les sources et les années 
display(pd.crosstab(df_clean1['company'], df_clean1['star']))
display(round(pd.crosstab(df_clean1['company'], df_clean1['star'], normalize=0), 2))

# affichage sur un graphique la distribution des star pour chaque company
sns.countplot(x='company', data=df_clean1, hue='star');

In [ ]:
# on peut faire de même avec la répartition de l'attribution des étoiles selon les sources
display(pd.crosstab(df_clean1['source'], df_clean1['star']))
display(round(pd.crosstab(df_clean1['source'], df_clean1['star'], normalize=0), 2))

# affichage sur un graphique la distribution des star pour chaque source
sns.countplot(x='source', data=df_clean1, hue='star');

In [ ]:
df.groupby(by=['source', 'company', 'star']).agg({'star' : 'count'})